In [ ]:
import os, colorcet, param as pm, holoviews as hv, panel as pn, datashader as ds
from holoviews.element import tiles as hvts
from holoviews.operation.datashader import rasterize, shade, spread
import holoviews.operation.datashader as hd
from collections import OrderedDict as odict
import geoviews
from cartopy import crs
import pandas as pd
from astropy.coordinates import Angle
import astropy.units as u

geoviews.extension('bokeh', logo=False)
pn.extension()

from holoviews import opts
from holoviews.plotting.links import DataLink
from astropy.time import Time
import datetime as dt
import numpy as np

from bokeh import models
from bokeh.models import ColumnDataSource, HoverTool
from holoviews.streams import RangeXY
from holoviews.selection import link_selections
from holoviews.operation import decimate

In [ ]:
filename = "/users/elowe/repos/antenna-plots/modified_ant_pos_17AB_plus.parquet"

# dataframes for each data set
df = pd.read_parquet(filename)
df = df[::10]
df

In [ ]:
%%time

# get unique list of receivers and backends
recs_list=df['Receiver'].unique().tolist()
backs_list = df['Backend'].unique().tolist()

cmaps  = odict([(n,colorcet.palette[n]) for n in ['fire', 'rainbow4', 'bmy', 'gray', 'kbc', 'blues']])
bg_colors  = {color.capitalize(): color for color in ["grey", "black", "white", "grey"]}
recs = {receiver: receiver for receiver in recs_list}
backends = {backends: backends for backends in backs_list}

# adding the option of All to the receiver
updict = {"All" : 'All'}

# ** operator for packing and unpacking items in order
recs = {**updict, **recs}

# Made the tools set
my_tools =['box_select','lasso_select','help']

# creating the pairing of backends to the reciever
pairingBackends = {}
for rec in recs:
    selectedReceiver = df[df['Receiver']==rec]
    backendOptions = selectedReceiver['Backend'].drop_duplicates()
    backendOptions = backendOptions.tolist()
    backendOptions.insert(0,'All')
    pairingBackends[rec] = backendOptions
pairingBackends['All'] = ['All',*backs_list]

#get unique list of sessions
sess_list=df['Session'].unique().tolist()
sess_list.sort()
sess_list.insert(0,'All')

In [ ]:
# Create all the widgets 
background_color = pn.widgets.Select(name="Background Color", value='grey', options=bg_colors)
cmap = pn.widgets.Select(name="Cmap Color", value='fire', options=cmaps)
spread_threshold = pn.widgets.FloatSlider(name='Spread Threshold', start=0, end=1, step=.1, value=.4)
spread_max_pixels = pn.widgets.IntSlider(name='Spread Max Pixels', start=0, end=10, step=1, value=1)
ra_rotation = pn.widgets.IntSlider(name='RA Rotation (Degrees)', start=-180, end=180, step=1, value=0)
dec_rotation = pn.widgets.IntSlider(name='Dec Rotation (Degrees)', start=-180, end=180, step=1, value=0)
receiver_selection = pn.widgets.Select(name='Receivers',value='All',options=recs)
backend_selection = pn.widgets.MultiSelect(name='Backends', value=['All'],options=pairingBackends[receiver_selection.value],size = 8)
session_selection = pn.widgets.MultiSelect(name='Session', value=['All'],options=sess_list,size = 8)
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=Time(df["DMJD"].min(), format="mjd").to_datetime(), end=Time(df["DMJD"].max(), format="mjd").to_datetime(),
    value=(Time(df["DMJD"].min(), format="mjd").to_datetime(), Time(df["DMJD"].max(), format="mjd").to_datetime()),
)

# widgets to hold the place for the projections
ra_dec = pn.widgets
gal_ra_dec = pn.widgets
azi_el =pn.widgets

# List of all the widgets
widgets = [
    background_color,
    cmap,
    spread_threshold,
    spread_max_pixels,
    ra_rotation,
    dec_rotation,
    date_range_slider,
    session_selection,
    receiver_selection,
    backend_selection
]

# Add all the widgets to the WidgetBox
option_box = pn.WidgetBox(*widgets)

In [ ]:
# Add all three projections into a column
my_column = pn.Column(
    pn.Column('# RA/Dec J2000', ra_dec),
    pn.Column('# Galactic Ra/Dec', gal_ra_dec),
    pn.Column('# Az/El', azi_el),
)

# create dict of values for projections
refs = {0:['RAJ2000','DECJ2000', 400,crs.Mollweide()],
        1:['gal_ra','gal_dec',400,crs.Mollweide()],
        2:['MNT_AZ','MNT_EL',800,crs.AzimuthalEquidistant()]}

# dataset and table link for linking the projections to the table
dataset = hv.Dataset(df)
table_link = link_selections.instance()
    
    
# Creating the plots
def update_proj(event):
    
    # lists that hold the points and the projections
    list_new_points = []
    list_new_projected = []

    # Loop through and make each plot
    for i in range(3):
        
        # create condtions to check
        isValidDateRange = (date_range_slider.value_start <  df['date']) & (df['date'] < date_range_slider.value_end) 
        isValidReceiver = receiver_selection.value=='All' or df['Receiver']==receiver_selection.value
        isValidBackend = backend_selection.value==['All'] or df['Backend'].isin(backend_selection.value)
        session_select = session_selection.value==['All'] or df['Session'].isin(session_selection.value)
        
        # create mask that will only have the points in the valid range
        mask = session_select &  isValidDateRange & isValidReceiver & isValidBackend

        # get new points based on rotation
        ra_rotated = df[refs[i][0]].where(mask) + ra_rotation.value
        ra_rotated[ra_rotated > 180] -= 360

        dec_rotated = df[refs[i][1]].where(mask) + dec_rotation.value
        dec_rotated[dec_rotated > 90] -= 180

        # New dataframe with the new positions
        new_df = pd.DataFrame({refs[i][0]: ra_rotated, refs[i][1]: dec_rotated})
        
        new_points = geoviews.Points(new_df, [refs[i][0], refs[i][1]])
        new_points = new_points.opts(opts.Points(tools=['box_select', 'lasso_select'],projection=refs[i][3], global_extent=True, width=800, height=refs[i][2]))
        new_projected = geoviews.operation.project_points(new_points, projection=refs[i][3])
        
         # creating the decimated points
        decimated_points = decimate(new_projected).opts(tools=['lasso_select', 'box_select'], projection=refs[i][3], alpha=0)
        
        # Shading the data
        hv_shaded = hv.operation.datashader.datashade(rasterize(hv.DynamicMap(new_projected)), dynamic=True, precompute=True, cmap=cmap.value)
        hv_spread = hv.operation.datashader.dynspread(hv_shaded, threshold=spread_threshold.value, max_px=spread_max_pixels.value)
        hv_spread = hv_spread.opts(tools=['box_select','lasso_select','help'], bgcolor=background_color.value,projection=refs[i][3], global_extent=True, frame_width=800, frame_height=refs[i][2])
        
        # for the quadmesh
        dynamic = hv.util.Dynamic(hd.aggregate(new_projected, width=12, height=12, streams=[RangeXY]),operation=hv.QuadMesh).opts(tools=['hover'], projection=refs[i][3],alpha=0, hover_alpha=0.9)
        
        my_column[i][1] = (hv_spread * decimated_points * dynamic)
         

# creates the selection table
@pm.depends(table_link.param.selection_expr)
def selection_table(_):
    return hv.Table(dataset.select(table_link.selection_expr)).opts(width=900, height=200)  

# the method that populates the beckends based on the receiver selected
@pn.depends(receiver_selection.param.value, watch=True)
def _update_backend(receiver_selection):
    backendList = pairingBackends[receiver_selection]
    backend_selection.options = backendList
    valueList = [backendList[0]]
    backend_selection.value = valueList
    
# To loop through all the widgets  
def add_trigger(widget):
    widget.param.watch(update_proj, 'value')

# position.param.watch(update_proj, 'value')
# position.param.trigger('value')
# replaced by:
# widget.param.trigger('value')
for widget in widgets:
    add_trigger(widget)

# GBO logo 
logo = "https://i0.wp.com/greenbankobservatory.org/wp-content/uploads/2016/10/logo_purp-green_grad.png"


# Adding them to a row
pn.Row(
    pn.Column(logo,
    option_box),
    my_column #selection_table
)